In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import nibabel as nib
import tqdm
from tqdm import tqdm_notebook

# fMRI data preparation

# Data Separation

In [ ]:
basedir = '/users/jsh/data/IG/ICA/2023-04-25/input' # Input path
targetdir = '/users/jsh/data/IG/ICA/2023-04-25' # Save path

In [ ]:
sbj_list = pd.read_csv('/users/jsh/data/IG/ICA/2023-04-25/result/first_para_ica_sel_data.txt', header = None)
sbj_list.columns = ['temp']
sbj_list = sbj_list.loc[0:len(sbj_list)/2]
sbj_list['sbj'] = sbj_list.temp.str.split('/').str[9]
sbj_list['case'] = sbj_list.temp.str.split('/').str[8]
sbj_list = sbj_list.dropna(axis = 0)
sbj_info = sbj_list[['sbj', 'case']]

temp = []

for i in sbj_info['sbj']:
    temp.append(i[:4] + '_' + i[4:])
    
sbj_info['subjectkey'] = temp


data = pd.read_csv('/users/jsh/data/IG/Demographic+Behavior/abcd_sst02.txt', sep = '\t')

data = data.drop(['collection_id', 'abcd_sst02_id', 'dataset_id', 'src_subject_id', 'interview_date', 'collection_title'], axis = 1)
data = data.loc[data['eventname'] == 'baseline_year_1_arm_1']
data = data.drop(['eventname'], axis = 1)

data_merged = pd.merge(sbj_info, data, how = 'inner', on = 'subjectkey')

scanner = pd.read_csv('/users/jsh/data/IG/Demographic+Behavior/abcd_mri01.txt', sep = '\t')
scanner = scanner.loc[scanner['subjectkey'].isin(data_merged['subjectkey'].tolist())]
scanner = scanner[['subjectkey', 'mri_info_manufacturer']]

data_final = pd.merge(data_merged, scanner, on = 'subjectkey')

In [ ]:
HC = data_final.loc[data_final['case'] == 'control']
ADHD = data_final.loc[data_final['case'] == 'ADHD']

HC_train, HC_test = train_test_split(HC, test_size = 0.2, random_state = 0, stratify = HC[['sex', 'mri_info_manufacturer']])
ADHD_train, ADHD_test = train_test_split(ADHD, test_size = 0.2, random_state = 0, stratify = ADHD[['sex', 'mri_info_manufacturer']])

print(HC_train.shape)
print(HC_test.shape)
print(ADHD_train.shape)
print(ADHD_test.shape)

In [ ]:
# Train

basedir = '/users/jsh/data/IG/ICA/2023-04-25/input'
targetdir = '/users/jsh/data/IG/ICA/2023-04-25/input_origin'

for i in tqdm.tqdm_notebook(HC_train['sbj'].tolist()):
    os.system('cp -r {}/control/{} {}/control'.format(basedir, i, targetdir))
    
for i in tqdm.tqdm_notebook(ADHD_train['sbj'].tolist()):
    os.system('cp -r {}/ADHD/{} {}/ADHD'.format(basedir, i, targetdir))
    

# Test

basedir = '/users/jsh/data/IG/ICA/2023-04-25/input'
targetdir = '/users/jsh/data/IG/ICA/2023-04-25/input_valid'

for i in tqdm.tqdm_notebook(HC_test['sbj'].tolist()):
    os.system('cp -r {}/control/{} {}/control'.format(basedir, i, targetdir))
    
for i in tqdm.tqdm_notebook(ADHD_test['sbj'].tolist()):
    os.system('cp -r {}/ADHD/{} {}/ADHD'.format(basedir, i, targetdir))